Global setup



In [6]:
n=1
k=4
q=41 #Alkaline D
#q should be divisible by 8 for NTT
eta=1 #secret key range, prefered 2 in Alkaline
gamma1=2^3 #y coeff range, power of 2, exponent about 2/3 of q?  Earlier version was *not* power of 2

In [7]:
clenbits = log(n*k,2)
clenbits

2

In [8]:
tau = 1 #number of 1's in c
ln(binomial(4, tau)).n()+tau

2.38629436111989

In [9]:
beta = tau*eta
beta

1

Chance of $\mathbf{y}_i$-coefficient passing:

In [4]:
((2*(gamma1-beta)-1)/(2*gamma1-1))^(n*k).n()

0.564167901234568

Expected number of repetitions:

In [5]:
1/(((2*(gamma1-beta)-1)/(2*gamma1-1))^(n*k))^2.n()

3.14183413597341

Alice generates a signing key

In [6]:
#A = random_matrix(ZZ, 4, 4, x=0, y=q)
A = matrix(4,4, 
[[ 4, 14,  4, 14],
[27, 28,  8, 40],
[ 9,  5,  8, 19],
[ 8,  9, 14,  3]])
A

[ 4 14  4 14]
[27 28  8 40]
[ 9  5  8 19]
[ 8  9 14  3]

In [7]:
latex(A)

\left(\begin{array}{rrrr}
4 & 14 & 4 & 14 \\
27 & 28 & 8 & 40 \\
9 & 5 & 8 & 19 \\
8 & 9 & 14 & 3
\end{array}\right)

In [8]:
#S1 = random_matrix(ZZ, 4, 4, x=-eta, y=eta+1)
S1 = matrix(ZZ, 
[[-1,  0, -1,  1],
[ 0,  1,  0,  1],
[ 1, -1,  0,  1],
[-1,  0,  0,  0]])
S1

[-1  0 -1  1]
[ 0  1  0  1]
[ 1 -1  0  1]
[-1  0  0  0]

In [9]:
#S2 = random_matrix(ZZ, 4, 4, x=-eta, y=eta+1)
S2 = matrix(ZZ, 
[[ 0, -1, -1,  0],
[ 1, -1,  0, -1],
[ 0, -1,  1,  1],
[ 0,  1,  0, -1]])
S2

[ 0 -1 -1  0]
[ 1 -1  0 -1]
[ 0 -1  1  1]
[ 0  1  0 -1]

$(S_1, S_2)$ is the private key


In [10]:
T = matrix(((A*S1 + S2) % q))
T

[27  9 36 22]
[24 19 14 21]
[21 37 33 23]
[ 3 37 33 30]

In [11]:
latex(T)

\left(\begin{array}{rrrr}
27 & 9 & 36 & 22 \\
24 & 19 & 14 & 21 \\
21 & 37 & 33 & 23 \\
3 & 37 & 33 & 30
\end{array}\right)

$(A,T)$ is the public key


Alice signs the message



In [80]:
M = 'hi!'
M

'hi!'

In [120]:
#y1 = random_matrix(ZZ, 4, 1, x=-(gamma1-1), y=gamma1)
y1 = matrix(4, 1, [-7, 3, 1, 1])
y1

[-7]
[ 3]
[ 1]
[ 1]

In [121]:
#y2 = random_matrix(ZZ, 4, 1, x=-(gamma1-1), y=gamma1)
y2 = matrix(4, 1, [5, 0, -2, 4])
y2

[ 5]
[ 0]
[-2]
[ 4]

In [122]:
w1 = matrix(((A*y1+y2) % q)).apply_map(lambda r: Mod(r,q).lift_centered())
w1

[ -4]
[-16]
[ 18]
[ -8]

In [4]:
from sage.cpython.string import str_to_bytes

Mw1 = f"{M}<{w1[0][0]},{w1[1][0]},{w1[2][0]},{w1[3][0]}>"
str_to_bytes(Mw1)

NameError: name 'M' is not defined

In [3]:
import hashlib

c_hash = hashlib.shake_256(str_to_bytes(Mw1))

NameError: name 'str_to_bytes' is not defined

In [143]:
c_hash.hexdigest(n*k)

'c1c9a017'

In [144]:
int(c_hash.hexdigest(3*n*k)[2], 16)

12

SHA-1 for Maple compatibility :(

In [145]:
hashlib.sha1(str_to_bytes(Mw1)).hexdigest()

'830bb6cada00d4de7c82e38cd63d5f5a7f7611db'

In [146]:
format(int(hashlib.sha1(str_to_bytes(Mw1)).hexdigest(),16),f'0{160}b')

'1000001100001011101101101100101011011010000000001101010011011110011111001000001011100011100011001101011000111101010111110101101001111111011101100001000111011011'

In [10]:
nk = n*k
nk

4

In [130]:
tau

1

In [2]:
def sample_in_ball(digest):  # newer version; uses fewer bits
    c = [0] * nk
    k = tau
    for i in range(nk-tau, nk):
        while int(digest[k:k+clenbits], 2) > i:
            k = k+clenbits
        j = int(digest[k:k+clenbits], 2)
        c[i] = c[j]
        c[j] = (-1)^int(digest[i+tau-nk], 2)
        k = k+clenbits
    return c
    

In [139]:
c = matrix(4, 1, sample_in_ball(format(int(hashlib.sha1(str_to_bytes(Mw1)).hexdigest(),16),f'0{160}b')))
c

[-1]
[ 0]
[ 0]
[ 0]

In [133]:
z1 = matrix(y1 + S1*c); z1

[-6]
[ 3]
[ 0]
[ 2]

Is $\|\mathbf{z_1}\|_\infty \geq \gamma_1-\beta$?

In [134]:
gamma1-beta

7

If so, reject and pick a new $\mathbf{y_1}$

In [135]:
z2 = matrix(y2 + S2*c); z2

[ 5]
[-1]
[-2]
[ 4]

Is $\|\mathbf{z_2}\|_\infty \geq \gamma_1-\beta$?

In [136]:
gamma1-beta

7

If so, reject and pick a new $\mathbf{y_2}$

If not, $(\mathbf{z_1}, \mathbf{z_2}, \mathbf{c})$ is the signature


In [137]:
(z1, z2, c)

(
[-6]  [ 5]  [-1]
[ 3]  [-1]  [ 0]
[ 0]  [-2]  [ 0]
[ 2], [ 4], [ 0]
)

Bob verifies the signature

Is $\|\mathbf{z_i}\|_\infty < \gamma_1-\beta$?

In [114]:
z1

[-6]
[ 3]
[ 0]
[ 2]

In [115]:
z2

[ 5]
[-1]
[-2]
[ 4]

In [116]:
gamma1-beta

7

Yes

Is $c = H(M \| w_1')$?

In [117]:
w1prime = matrix(((A*z1+z2-T*c) % q)).apply_map(lambda r: Mod(r,q).lift_centered())
w1prime

[ -4]
[-16]
[ 18]
[ -8]

In [106]:
cprime_hash = hashlib.shake_256(str_to_bytes(M+str(w1prime)))
cprime = matrix(4, 1, sample_in_ball(cprime_hash.hexdigest(n*k)))
cprime

[ 0]
[-1]
[ 0]
[ 0]

In [118]:
cprime = matrix(4, 1, sample_in_ball(format(int(hashlib.sha1(str_to_bytes(Mw1)).hexdigest(),16),f'0{160}b')))
cprime

[-1]
[ 0]
[ 0]
[ 0]

In [119]:
c == cprime

True

Yes.  The signature is accepted.

Another SHA-1 test

In [13]:
c = matrix(4, 1, sample_in_ball(format(int(hashlib.sha1(str_to_bytes("hi!<-4,-16,18,-8>")).hexdigest(),16),f'0{160}b')))
c

[-1]
[ 0]
[ 0]
[ 0]

In [35]:
#from sage.cpython.sys import int_from_bytes

bin(int.from_bytes(str_to_bytes("hi!<-4,-16,18,-8>"), byteorder=sys.byteorder))

'0b11111000111000001011010010110000111000001100010010110000110110001100010010110100101100001101000010110100111100001000010110100101101000'

In [22]:
nk = 6
tau = 2
c = matrix(6, 1, sample_in_ball(format(int(hashlib.sha1(str_to_bytes("another test")).hexdigest(),16),f'0{160}b')))
c

[ 0]
[ 0]
[-1]
[ 1]
[ 0]
[ 0]